In [1]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score  # <--- Added thi

In [2]:
df = pd.read_csv('data_solar.csv')

In [3]:

df["season_sin"] = np.sin(2 * np.pi * df["Season"] / 4)
df["season_cos"] = np.cos(2 * np.pi * df["Season"] / 4)
df["weekday_sin"] = np.sin(2 * np.pi * df["Day_of_the_week"] / 7)
df["weekday_cos"] = np.cos(2 * np.pi * df["Day_of_the_week"] / 7)
df["PV_potential"] = df["GHI"]*df["Temperature"]
df.drop(columns=['Season','Day_of_the_week'],inplace=True)
df['Time'] = pd.to_datetime(df['Time'], format='%Y-%m-%d-T%H:%M')
df['GHI_diff_5m'] = df['GHI'].diff(periods=1)
df['GHI_rolling_std_30m'] = df['GHI'].rolling(window=6).std()
df['hour_min'] = df['Time'].dt.strftime('%H:%M')
clear_sky_proxy = df.groupby('hour_min')['GHI'].transform('max')
df['Clearness_Index'] = df['GHI'] / (clear_sky_proxy + 1e-9) # Avoid div by zero
df.dropna(inplace=True)
df.drop(columns=['hour_min'],inplace=True)
df['target'] = df['PV_production'] 
df.drop(columns=['PV_production'],inplace=True)
df['target'] = df['target'].clip(0,1)


In [4]:
df.to_csv("data_solar_featured.csv",index=False)

In [15]:
minmax_cols = [
    'DHI', 'DNI', 'GHI', 'Wind_speed', 'Humidity', 'Temperature',
    
    'PV_potential', 'GHI_diff_5m', 'GHI_rolling_std_30m', 'Clearness_Index'
]
LOOKBACK_STEPS = 12            # 1 hour history
BATCH_SIZE = 32
EPOCHS = 8
LEARNING_RATE = 0.001
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(f"Using device: {DEVICE}")

Using device: cpu


In [6]:
feature_cols = [
    'DHI', 'DNI', 'GHI', 'Wind_speed', 'Humidity', 'Temperature',
    'season_sin', 'season_cos', 'weekday_sin', 'weekday_cos',
    'PV_potential', 'GHI_diff_5m', 'GHI_rolling_std_30m', 'Clearness_Index'
]

In [7]:
scaler = MinMaxScaler()
df[minmax_cols] = scaler.fit_transform(df[minmax_cols])
X_values = df[feature_cols].values.astype(np.float32)
y_values = df['target'].values.astype(np.float32)

In [8]:
class SolarDataset(Dataset):
    def __init__(self, features, targets, lookback):
        self.features = features
        self.targets = targets
        self.lookback = lookback
        
    def __len__(self):
        return len(self.features) - self.lookback

    def __getitem__(self, idx):
        # Sequence of 'lookback' steps
        x = self.features[idx : idx + self.lookback]
        # Target is the NEXT step
        y = self.targets[idx + self.lookback]
        return torch.tensor(x), torch.tensor(y).unsqueeze(-1) # Add dimension for shape (1)

In [9]:
# --- 4. SPLIT & DATALOADERS ---
split_idx = int(len(X_values) * 0.9)

train_dataset = SolarDataset(X_values[:split_idx], y_values[:split_idx], LOOKBACK_STEPS)
test_dataset = SolarDataset(X_values[split_idx:], y_values[split_idx:], LOOKBACK_STEPS)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=False) # Time series: shuffle=False usually
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [10]:
batch = next(iter(train_loader))
X, y = batch
print("X shape:", X.shape)
print("y shape:", y.shape)


X shape: torch.Size([32, 12, 14])
y shape: torch.Size([32, 1])


In [11]:
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size=64, num_layers=2, dropout=0.2):
        super(LSTMModel, self).__init__()
        
        # LSTM Layer
        self.lstm = nn.LSTM(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout
        )
        
        # Fully Connected Layer
        self.fc = nn.Linear(hidden_size, 1)
        
        # Activation for [0, 1] output
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        # x shape: (batch, seq_len, features)
        # out shape: (batch, seq_len, hidden)
        out, _ = self.lstm(x)
        
        # We only care about the last time step's output
        last_step_out = out[:, -1, :] 
        
        prediction = self.fc(last_step_out)
        return self.sigmoid(prediction)

In [12]:


input_dim = len(feature_cols)
print(input_dim)
model = LSTMModel(input_size=input_dim).to(DEVICE)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

14


In [16]:
from tqdm import tqdm
import os # To help with file path handling if needed

# --- CONFIGURATION FOR SAVING ---
MODEL_SAVE_PATH = "best_solar_model.pth"
best_val_r2 = -float('inf') # Initialize with negative infinity

print(f"Starting Training on {DEVICE}...")

for epoch in range(EPOCHS):
    # 1. Training Phase
    model.train()
    train_loss = 0
    
    loop = tqdm(train_loader, desc=f"Epoch [{epoch+1}/{EPOCHS}]", leave=True)
    
    for batch_X, batch_y in loop:
        batch_X, batch_y = batch_X.to(DEVICE), batch_y.to(DEVICE)
        
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
        loop.set_postfix(loss=loss.item())
    
    # 2. Validation Phase
    model.eval()
    val_preds = []
    val_targets = []
    
    with torch.no_grad():
        for batch_X, batch_y in test_loader:
            batch_X = batch_X.to(DEVICE)
            outputs = model(batch_X)
            val_preds.append(outputs.cpu().numpy())
            val_targets.append(batch_y.cpu().numpy())
            
    # Calculate Scores
    val_preds = np.concatenate(val_preds)
    val_targets = np.concatenate(val_targets)
    
    val_r2 = r2_score(val_targets, val_preds)
    avg_train_loss = train_loss / len(train_loader)
    
    # Print Epoch Summary
    print(f"Epoch [{epoch+1}/{EPOCHS}] | Train MSE: {avg_train_loss:.5f} | Val R2 Score: {val_r2:.4f}")

    # --- SAVE MODEL IF R2 IMPROVES ---
    if val_r2 > best_val_r2:
        print(f"🔥 R2 improved from {best_val_r2:.4f} to {val_r2:.4f}. Saving model...")
        best_val_r2 = val_r2
        torch.save(model.state_dict(), MODEL_SAVE_PATH)

print(f"Training Complete. Best Model R2: {best_val_r2:.4f} saved to {MODEL_SAVE_PATH}")

Starting Training on cpu...


Epoch [1/8]: 100%|██████████| 8878/8878 [02:30<00:00, 58.99it/s, loss=0.00137]  


Epoch [1/8] | Train MSE: 0.00489 | Val R2 Score: 0.8913
🔥 R2 improved from -inf to 0.8913. Saving model...


Epoch [2/8]: 100%|██████████| 8878/8878 [02:08<00:00, 69.17it/s, loss=0.00171]  


Epoch [2/8] | Train MSE: 0.00372 | Val R2 Score: 0.8946
🔥 R2 improved from 0.8913 to 0.8946. Saving model...


...

Epoch [6/8] | Train MSE: 0.00321 | Val R2 Score: 0.9132
🔥 R2 improved from 0.9119 to 0.9132. Saving model...


Epoch [7/8]: 100%|██████████| 8878/8878 [01:48<00:00, 81.91it/s, loss=0.00182]  


Epoch [7/8] | Train MSE: 0.00315 | Val R2 Score: 0.9141
🔥 R2 improved from 0.9132 to 0.9141. Saving model...


Epoch [8/8]: 100%|██████████| 8878/8878 [02:13<00:00, 66.71it/s, loss=0.0019]  


Epoch [8/8] | Train MSE: 0.00311 | Val R2 Score: 0.9148
🔥 R2 improved from 0.9141 to 0.9148. Saving model...
Training Complete. Best Model R2: 0.9148 saved to best_solar_model.pth


In [19]:
import joblib
import pandas as pd

# 1. Save the Scaler (CRITICAL)
joblib.dump(scaler, 'scaler.gz')

# 2. Create and Save the "Clear Sky" Lookup Map
# (We need this to calculate Clearness_Index for new data)
df['hour_min'] = df['Time'].dt.strftime('%H:%M')
max_ghi_map = df.groupby('hour_min')['GHI'].max().to_dict()
joblib.dump(max_ghi_map, 'max_ghi_map.gz')

print("✅ Artifacts saved: scaler.gz and max_ghi_map.gz")

✅ Artifacts saved: scaler.gz and max_ghi_map.gz


In [18]:
import matplotlib.pyplot as plt
import seaborn as sns

# --- 1. PREPARE DATA FOR PLOTTING ---
# Ensure model is in eval mode and grab a slice of data
model.eval()

# Let's visualize the LAST 300 data points (approx 1 day of 5-min intervals)
# or 1000 points (approx 3.5 days) to show day/night cycles clearly.
PLOT_LENGTH = 1000 

# Get predictions
with torch.no_grad():
    # Take the last 'PLOT_LENGTH' samples from the test loader (simplified approach)
    # Ideally, grab from your full X_test array you created earlier
    test_X_slice = torch.tensor(X_test[-PLOT_LENGTH:]).to(DEVICE)
    test_y_slice = y_test[-PLOT_LENGTH:] # This is already numpy array from your split
    
    # Predict
    preds_slice = model(test_X_slice).cpu().numpy()

# --- DENORMALIZE (CRITICAL STEP) ---
# Convert 0-1 back to Real MW
# Assuming MAX_INSTALLED_CAPACITY is defined (e.g., 3000 MW)
actual_mw = test_y_slice * MAX_INSTALLED_CAPACITY
pred_mw = preds_slice * MAX_INSTALLED_CAPACITY

# Calculate Error in MW
error_mw = actual_mw - pred_mw

# --- 2. PLOT 1: THE "REALISM" PLOT (Actual vs Predicted) ---
plt.figure(figsize=(15, 6))
plt.plot(actual_mw, label='Actual Generation (MW)', color='black', alpha=0.6, linewidth=2)
plt.plot(pred_mw, label='AI Prediction (MW)', color='#00d2be', linestyle='--', linewidth=2) # Cyan/Teal pops on slides

plt.title(f"Real-Time Forecast vs Actual (Last {PLOT_LENGTH/12/24:.1f} Days)", fontsize=16, fontweight='bold')
plt.ylabel("Power Generation (MW)", fontsize=12)
plt.xlabel("Time Steps (5-min intervals)", fontsize=12)
plt.legend(fontsize=12)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# --- 3. PLOT 2: THE "GRID STABILITY" PLOT (Error Volume) ---
# This looks much better than a line at zero. It shows "Unbalanced Power".
plt.figure(figsize=(15, 4))

# Fill area: Red for Under-prediction (Grid stress), Blue for Over-prediction (Curtailment)
plt.fill_between(range(len(error_mw)), error_mw.flatten(), 0, 
                 where=(error_mw.flatten() >= 0), color='firebrick', alpha=0.5, label='Under-Forecasting (Need Reserves)')
plt.fill_between(range(len(error_mw)), error_mw.flatten(), 0, 
                 where=(error_mw.flatten() < 0), color='royalblue', alpha=0.5, label='Over-Forecasting (Surplus)')

plt.title("Forecast Error Deviation (MW) - Stability Analysis", fontsize=16, fontweight='bold')
plt.ylabel("Error (MW)", fontsize=12)
plt.xlabel("Time Steps", fontsize=12)
plt.legend(loc='upper right')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# --- 4. PLOT 3: THE "R2 PROOF" PLOT (Scatter) ---
plt.figure(figsize=(8, 8))
plt.scatter(actual_mw, pred_mw, alpha=0.3, s=10, color='purple')

# Perfect prediction line
p1 = max(max(pred_mw), max(actual_mw))
p2 = min(min(pred_mw), min(actual_mw))
plt.plot([p1, p2], [p1, p2], 'r--', linewidth=2, label='Perfect Prediction')

plt.title(f"Model Fit Accuracy (R2 Score: {val_r2:.4f})", fontsize=16, fontweight='bold')
plt.xlabel("Actual MW", fontsize=12)
plt.ylabel("Predicted MW", fontsize=12)
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

NameError: name 'X_test' is not defined